In [1]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
import torch
import pyvene as pv
import pprint
from task import MetaLearningTask
from transformers import BatchEncoding, PretrainedConfig
import jax.random as jr
from torch2jax import j2t, t2j
import dataclasses

2024-10-23 14:36:23.716303: W external/xla/xla/service/gpu/nvptx_compiler.cc:893] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.6.68. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda'

In [2]:
task = MetaLearningTask('b9h7hygi')
task : MetaLearningTask
task.eval();
task.model.to('cuda');

number of parameters: 18.94M
Loaded dataset : (11288/1000)
Loaded checkpoing : epoch=438,step=19750.ckpt


In [3]:
task.model.config = PretrainedConfig.from_dict(dataclasses.asdict(task.model.config))

In [4]:
collect_model = pv.IntervenableModel(
    [
        {
            "component": f"transformer.h[{l}].output",
            "intervention": pv.CollectIntervention(),
        }
        for l in range(2, 5)
    ],
    model=task.model,
)

In [5]:
seq = j2t(task.full_data.sample(0,100,jr.PRNGKey(0)))

In [15]:
acts = collect_model(
    base=BatchEncoding({'input_ids': seq[None]}),
    unit_locations={"base": 99},
    return_dict=True,
)['collected_activations']

In [13]:
list(res.values())[0].shape

torch.Size([1, 1, 512])

In [16]:
intv_model = pv.IntervenableModel(
    [
        {
            "component": f"transformer.h[{l}].output",
            "intervention": pv.AdditionIntervention(source_representation=list(acts.values())[i][0,0]),
        }
        for i,l in enumerate(range(2,5))
    ],
    model=task.model
)

In [19]:
seq_b = j2t(task.full_data.sample(5432,100,jr.PRNGKey(0)))

In [24]:
res = intv_model.generate(
    base=BatchEncoding({'input_ids': seq_b[None]}),
    #unit_locations={"sources->base": (0, 3)},
    intervene_on_prompt=False,
    timestep_selector=[lambda idx, o: True for i in range(3)],
    #output_original_output=True,
    max_length=200
)

In [27]:
res[1].shape

torch.Size([1, 300])